
<a href="https://colab.research.google.com/github/google-research/bigbird/blob/master/bigbird/summarization/eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 The BigBird Authors

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2020 The BigBird Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

In [ ]:
!pip install git+https://github.com/google-research/bigbird.git -q

     |████████████████████████████████| 1.2MB 22.4MB/s 
     |████████████████████████████████| 3.4MB 23.8MB/s 
     |████████████████████████████████| 1.5MB 55.3MB/s 
     |████████████████████████████████| 3.8MB 54.1MB/s 
     |████████████████████████████████| 368kB 54.2MB/s 
     |████████████████████████████████| 368kB 26.4MB/s 
     |████████████████████████████████| 358kB 52.5MB/s 
     |████████████████████████████████| 983kB 58.2MB/s 
     |████████████████████████████████| 5.6MB 52.5MB/s 
     |████████████████████████████████| 378kB 52.1MB/s 
     |████████████████████████████████| 706kB 45.3MB/s 
     |████████████████████████████████| 194kB 58.8MB/s 
     |████████████████████████████████| 655kB 53.9MB/s 
     |████████████████████████████████| 163kB 51.4MB/s 
     |████████████████████████████████| 256kB 62.4MB/s 
ERROR: kfac 0.2.3 has requirement tensorflow-probability==0.8, but you'll have tensorflow-probability 0.7.0 which is incompatible.


In [ ]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
import tensorflow_text as tft
from tqdm import tqdm

tf.enable_v2_behavior()

## Load Saved Model

In [ ]:
path = 'gs://bigbird-transformer/summarization/pubmed/roberta/saved_model'
imported_model = tf.saved_model.load(path, tags='serve')
summerize = imported_model.signatures['serving_default']

## Setup Data

In [ ]:
dataset = tfds.load('cnn_dailymail', split='test', shuffle_files=False, as_supervised=True)

Dataset cnn_dailymail downloaded and prepared to /root/tensorflow_datasets/cnn_dailymail/3.1.0. Subsequent calls will reuse this data.


In [ ]:
# inspect at a few examples
for ex in dataset.take(3):
  print(ex)

(<tf.Tensor: shape=(), dtype=string, numpy=b"Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee.\xc2\xa0'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes 

## Print predictions

In [ ]:
predicted_summary = summerize(ex[0])['pred_sent'][0]

In [ ]:
print('Article:\n {}\n\n Predicted summary:\n {}\n\n Ground truth summary:\n {}\n\n'.format(
    ex[0].numpy(),
    predicted_summary.numpy(),
    ex[1].numpy()))

Article:
 b"Dougie Freedman is on the verge of agreeing a new two-year deal to remain at Nottingham Forest. Freedman has stabilised Forest since he replaced cult hero Stuart Pearce and the club's owners are pleased with the job he has done at the City Ground. Dougie Freedman is set to sign a new deal at Nottingham Forest . Freedman has impressed at the City Ground since replacing Stuart Pearce in February . They made an audacious attempt on the play-off places when Freedman replaced Pearce but have tailed off in recent weeks. That has not prevented Forest's ownership making moves to secure Freedman on a contract for the next two seasons."

 Predicted summary:
 b'this article is a series of juxtaposition .  \xe2\x81\x87  it is on the verge of agreeing a new two- and - three - year look .  \xe2\x81\x87  it is on the verge of having a new two- and - three - year look .  \xe2\x81\x87  juxtaposition .  \xe2\x81\x87  juxtaposition is on the verge of having a new two- and - three - year look 

## Evaluate Rouge Scores

In [ ]:
from rouge_score import rouge_scorer
from rouge_score import scoring

In [ ]:
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeLsum"], use_stemmer=True)
aggregator = scoring.BootstrapAggregator()

In [ ]:
for ex in tqdm(dataset.take(100), position=0):
  predicted_summary = summerize(ex[0])['pred_sent'][0]
  score = scorer.score(ex[1].numpy().decode('utf-8'), predicted_summary.numpy().decode('utf-8'))
  aggregator.add_scores(score)

100%|██████████| 100/100 [38:52<00:00, 23.32s/it]


In [ ]:
aggregator.aggregate()

{'rouge1': AggregateScore(low=Score(precision=0.0626133716965276, recall=0.09245324354755759, fmeasure=0.06909132433045363), mid=Score(precision=0.0746583443146113, recall=0.10755899354204748, fmeasure=0.08052101710380298), high=Score(precision=0.08904032679502548, recall=0.1236824852479251, fmeasure=0.0928306329324628)),
 'rouge2': AggregateScore(low=Score(precision=0.00381175250519571, recall=0.005956829545162127, fmeasure=0.00438814247007295), mid=Score(precision=0.005865650668740157, recall=0.008746112948814094, fmeasure=0.00665157835069642), high=Score(precision=0.0084917200439844, recall=0.012320302379654698, fmeasure=0.009338255444333433)),
 'rougeLsum': AggregateScore(low=Score(precision=0.05585761434091965, recall=0.0830323189114007, fmeasure=0.06221098171160909), mid=Score(precision=0.0666258100692349, recall=0.09613139230841955, fmeasure=0.07175521600863494), high=Score(precision=0.07954490388251197, recall=0.10937686014969702, fmeasure=0.08263920218454585))}